<a href="https://colab.research.google.com/github/LiamSwick/Data-Science-Sandbox/blob/main/TensorFlow_MNIST_Part1_with_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Neural Network for MNIST Classification

We'll apply all the knowledge from the lectures in this section to write a deep neural network. The problem we've chosen is referred to as the "Hello World" of deep learning because for most students it is the first deep learning algorithm they see.

The dataset is called MNIST and refers to handwritten digit recognition. You can find more about it on Yann LeCun's website (Director of AI Research, Facebook). He is one of the pioneers of what we've been talking about and of more complex approaches that are widely used today, such as covolutional neural networks (CNNs).

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image).

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes.

Our goal would be to build a neural network with 2 hidden layers.

## Import the relevant packages

In [56]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds


In [57]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [58]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [59]:
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

batch_size = 100
train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

In [60]:
input_size = 784
output_size = 10
hidden_layer_size = 200
model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ]

                            )

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [61]:
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [62]:
num_epochs = 5

model.fit(train_data, epochs=num_epochs, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 13s - 24ms/step - accuracy: 0.9249 - loss: 0.2549 - val_accuracy: 0.9653 - val_loss: 0.1249
Epoch 2/5
540/540 - 6s - 10ms/step - accuracy: 0.9702 - loss: 0.0971 - val_accuracy: 0.9760 - val_loss: 0.0802
Epoch 3/5
540/540 - 7s - 13ms/step - accuracy: 0.9802 - loss: 0.0650 - val_accuracy: 0.9817 - val_loss: 0.0609
Epoch 4/5
540/540 - 5s - 10ms/step - accuracy: 0.9846 - loss: 0.0486 - val_accuracy: 0.9870 - val_loss: 0.0483
Epoch 5/5
540/540 - 6s - 11ms/step - accuracy: 0.9884 - loss: 0.0371 - val_accuracy: 0.9890 - val_loss: 0.0390


In [63]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.9770 - loss: 0.0789


In [65]:
print(f"Test loss is: {test_loss:0.2}", f"Test accuracy is: {test_accuracy:0.3}")

Test loss is: 0.079 Test accuracy is: 0.977
